Where we convert the raw data into training and validation sets.
Manila Observatory will be the "test" set.

`aerosol_type` is the response variable

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os

# MICE (imputation)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import linear_model

# Feature Selection (VIF)
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

c:\Users\Hans\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
REF_CLUSTER_DIR = "raw_data/reference_sites/"

site2type = {
    "Solar_Village" : "MD",
    "Beijing" : "PD",
    "Mongu" : "BBD",
    "Alta_Floresta": "BBW",
    "GSFC" : "UI",
    "Chen-Kung_Univ" : "UID",
}

site_min_month = {
    "Solar_Village" : 3,
    "Beijing" : 1,
    "Mongu" : 8,
    "Alta_Floresta": 8,
    "GSFC" : 6,
    "Chen-Kung_Univ" : 1,
}

site_max_month = {
    "Solar_Village" : 7,
    "Beijing" : 12,
    "Mongu" : 11,
    "Alta_Floresta": 10,
    "GSFC" : 9,
    "Chen-Kung_Univ" : 12,
}



cols_to_keep = [
    'Site',
    'Date(dd:mm:yyyy)',
    'Angstrom_Exponent_440-870nm_from_Coincident_Input_AOD',
    'Extinction_Angstrom_Exponent_440-870nm-Total',
    'Single_Scattering_Albedo[440nm]',
    'Single_Scattering_Albedo[675nm]',
    'Single_Scattering_Albedo[870nm]',
    'Single_Scattering_Albedo[1020nm]',
    'Absorption_Angstrom_Exponent_440-870nm',
    'Refractive_Index-Real_Part[440nm]',
    'Refractive_Index-Real_Part[675nm]',
    'Refractive_Index-Real_Part[870nm]',
    'Refractive_Index-Real_Part[1020nm]',
    'Refractive_Index-Imaginary_Part[440nm]',
    'Refractive_Index-Imaginary_Part[675nm]',
    'Refractive_Index-Imaginary_Part[870nm]',
    'Refractive_Index-Imaginary_Part[1020nm]',
    'Asymmetry_Factor-Total[440nm]',
    'Asymmetry_Factor-Total[675nm]',
    'Asymmetry_Factor-Total[870nm]',
    'Asymmetry_Factor-Total[1020nm]',
    'Asymmetry_Factor-Fine[440nm]',
    'Asymmetry_Factor-Fine[675nm]',
    'Asymmetry_Factor-Fine[870nm]',
    'Asymmetry_Factor-Fine[1020nm]',
    'Asymmetry_Factor-Coarse[440nm]',
    'Asymmetry_Factor-Coarse[675nm]',
    'Asymmetry_Factor-Coarse[870nm]',
    'Asymmetry_Factor-Coarse[1020nm]',
    'Sphericity_Factor(%)',
    'Lidar_Ratio[440nm]',
    'Lidar_Ratio[675nm]',
    'Lidar_Ratio[870nm]',
    'Lidar_Ratio[1020nm]',
    'Depolarization_Ratio[440nm]',
    'Depolarization_Ratio[675nm]',
    'Depolarization_Ratio[870nm]',
    'Depolarization_Ratio[1020nm]',
]

In [3]:
df = pd.concat([
    pd.read_csv(
        REF_CLUSTER_DIR  + f, skiprows=6, na_values=-999
    )[cols_to_keep].assign(
        date = lambda x: pd.to_datetime(x["Date(dd:mm:yyyy)"], format=r"%d:%m:%Y"),
        month = lambda x: x.date.dt.month,
        aerosol_type = lambda x: x.Site.map(site2type),
        start_month = lambda x: x.Site.map(site_min_month),
        end_month = lambda x: x.Site.map(site_max_month),
    ).query("start_month <= month <= end_month")\
    .drop(columns=["Site", "Date(dd:mm:yyyy)", "month", "start_month", "end_month"])
    for f in os.listdir(REF_CLUSTER_DIR)
])

In [4]:
df.aerosol_type.value_counts()

MD     6162
UI     3093
PD     3088
BBD    1984
UID    1550
BBW    1060
Name: aerosol_type, dtype: int64

In [5]:
df["Single_Scattering_Albedo[440nm]"].isna().mean()

0.4823758634941253

# Train-Validation Split

Data must be split before doing imputation in order to avoid data leakage. To keep things simple, we'll stick with a simple 70-30 split.

In [6]:
np.random.seed(2023)
rand_idx = np.random.permutation(df.index)
train_idx = rand_idx[:int(0.7*len(rand_idx))]
valid_idx = rand_idx[int(0.7*len(rand_idx)):]

In [7]:
train_df = df.loc[train_idx]
valid_df = df.loc[valid_idx]

# MICE imputation

SMOTE is for balancing the classes, in our case what we need is a method for imputation. A common method to use (apart from filling the NANs with the mean) is Multivariate Imputation by Chained Equation (MICE).

Ref: https://towardsdatascience.com/imputing-missing-data-with-simple-and-advanced-techniques-f5c7b157fb87#:~:text=One%20way%20to%20impute%20missing,with%20the%20previously%20observed%20value.

We fit the MICE imputer on the training set and use the learned model for imputing the validation and test sets.

In [8]:
train_df.columns[train_df.isna().mean()==1]

Index(['Sphericity_Factor(%)'], dtype='object')

In [9]:
# remove Sphericity_Factor(%) because the entire column is null.
X_cols = [c for c in train_df.columns if c not in ["aerosol_type", "date", "Sphericity_Factor(%)"]]

In [10]:
mice_imputer = IterativeImputer(
    estimator=linear_model.BayesianRidge(), 
    n_nearest_features=None,
    imputation_order="ascending"    
).fit(
    train_df[X_cols]
)

c:\Users\Hans\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [11]:
train_df = pd.concat([
    train_df.reset_index()[["aerosol_type", "date"]],
    pd.DataFrame(
        mice_imputer.transform(train_df[X_cols]),
        columns=X_cols
    )
], axis=1)

# Normalize Data

In [12]:
train_mean = train_df[X_cols].mean()
train_std = train_df[X_cols].std()

In [13]:
train_df[X_cols] = (train_df[X_cols] - train_mean)/ train_std

# Feature Selection

A lot of the columns are highly correlated -- let's multicollinearity using Variance Inflation Factor (VIF). High VIF features are iteratively removed until all variables have VIF < 10.

In [14]:
def calculate_vif(df, cols_to_check:list) -> dict:
    X = df[cols_to_check].values
    return {col: variance_inflation_factor(X,i) for i,col in tqdm(list(enumerate(cols_to_check)))}
 
 
cols_to_check= X_cols.copy() #create a copy
#remove columns with high VIF
vif_values = calculate_vif(train_df, cols_to_check)
vif_df = pd.DataFrame(vif_values.values(), index=vif_values.keys())
while (vif_df>10).max().item():
    cols_to_check.remove(vif_df.idxmax().item())
    vif_values = calculate_vif(train_df, cols_to_check)
    vif_df = pd.DataFrame(vif_values.values(), index=vif_values.keys())

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
cols_to_use = ["aerosol_type", "date"] + list(vif_df.index)

In [16]:
cols_to_use

['aerosol_type',
 'date',
 'Single_Scattering_Albedo[675nm]',
 'Absorption_Angstrom_Exponent_440-870nm',
 'Refractive_Index-Real_Part[440nm]',
 'Refractive_Index-Real_Part[1020nm]',
 'Asymmetry_Factor-Total[440nm]',
 'Asymmetry_Factor-Total[1020nm]',
 'Asymmetry_Factor-Fine[440nm]',
 'Asymmetry_Factor-Fine[1020nm]',
 'Asymmetry_Factor-Coarse[1020nm]',
 'Lidar_Ratio[440nm]',
 'Lidar_Ratio[1020nm]',
 'Depolarization_Ratio[440nm]']

# Finalize Train and Validation Sets

In [17]:
train_df[cols_to_use].to_csv("train_set.csv", index=False)

In [18]:
valid_temp = pd.DataFrame(mice_imputer.transform(valid_df[X_cols]), columns=X_cols)
valid_temp = (valid_temp - train_mean)/train_std

In [19]:
pd.concat([
    valid_df[["aerosol_type", "date"]].reset_index(),
    valid_temp
], axis=1)[cols_to_use].to_csv("valid_set.csv", index=False)

# Test Set (Manila Observatory)

In [20]:
test_df = pd.read_csv(
    "./raw_data/20090101_20221231_Manila_Observatory.all", skiprows=6, na_values=-999
)[cols_to_keep].assign(
    date = lambda x: pd.to_datetime(x["Date(dd:mm:yyyy)"], format=r"%d:%m:%Y"),
).drop(columns=["Site", "Date(dd:mm:yyyy)"])

In [21]:
test_temp = pd.DataFrame(mice_imputer.transform(test_df[X_cols]), columns=X_cols)
test_temp = (test_temp - train_mean)/ train_std

In [22]:
test_cols_to_use = [c for c in cols_to_use if c != "aerosol_type"]

In [24]:
test_temp.assign(
    date = test_df.date.values
)[test_cols_to_use].to_csv("test_set.csv", index=False)